<a href="https://colab.research.google.com/github/lakshithakr/FYP/blob/main/Phi_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

##We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Data Process

In [5]:
ft_prompt = """You are a creative domain name generator and an analyzer, with a sound knowledge about how the creative name are formed according to requirement and language use. Below is an instruction that describes a task of either generating proper domain names or analyzing a given domain name, paired with an input that provides further context. Write a response that appropriately completes the request with a proper name suggestions or analysis.

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    #instructions = examples["instruction"]
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    # for instruction, input, output in zip(instructions, inputs, outputs):
    #     # Must add EOS_TOKEN, otherwise your generation will go on forever!
    #     text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
    #     texts.append(text)
    # return { "text" : texts, }
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = ft_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset

# Replace 'path_to_your_file.csv' with the actual file path
data = load_dataset('csv', data_files='/content/gdrive/My Drive/final_df_eng.csv')

# View the dataset
print(data)
dataset = data.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 105829
    })
})


Map:   0%|          | 0/105829 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'text'],
        num_rows: 105829
    })
})

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    dataset_text_field="text",
    max_seq_length=2048,  # Adjusted for long sequences in Phi 3
    dataset_num_proc=4,  # Use more processors if available
    packing=True,  # Packing can help speed up training with long sequences
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Adjust based on GPU memory
        gradient_accumulation_steps=16,  # Effective batch size = 64
        warmup_steps=500,  # Longer warmup for stability
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps=8,  # Train for approximately 2 epochs with 100k records
        learning_rate=1e-4,  # Smaller learning rate for stability
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,  # Log less frequently to save time
        optim="adamw_8bit",  # Memory-efficient optimizer
        weight_decay=0.01,
        lr_scheduler_type="cosine",  # Smooth learning rate decay
        seed=42,  # Ensure reproducibility
        output_dir="Phi-3-mini-4k-instruct_finetuned",  # Clear experiment directory
        report_to="none",  # Add 'wandb' or similar for experiment tracking
        push_to_hub=True,
    ),
)

In [10]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 22,088 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 16
\        /    Total batch size = 32 | Total steps = 690
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss
1,2.130400
2,2.143400
3,2.147000
4,2.169200
5,2.146800
6,2.191100
7,2.189900
8,2.171800
9,2.144900
10,2.170000


In [12]:
# Save the model
trainer.model.save_pretrained("model_phi3")

# Save the tokenizer (if applicable)
trainer.tokenizer.save_pretrained("tokenizer_phi3")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('tokenizer_phi3/tokenizer_config.json',
 'tokenizer_phi3/special_tokens_map.json',
 'tokenizer_phi3/tokenizer.model',
 'tokenizer_phi3/added_tokens.json',
 'tokenizer_phi3/tokenizer.json')

In [13]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Lakshitha98/Phi-3-mini-4k-instruct_finetuned/commit/1442d642053b9927caeda038e252c060caccdcd9', commit_message='End of training', commit_description='', oid='1442d642053b9927caeda038e252c060caccdcd9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Lakshitha98/Phi-3-mini-4k-instruct_finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Lakshitha98/Phi-3-mini-4k-instruct_finetuned'), pr_revision=None, pr_num=None)

In [14]:
# Save the model
trainer.model.save_pretrained("model_phi3")

# Save the tokenizer (if applicable)
trainer.tokenizer.save_pretrained("tokenizer_phi3")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('tokenizer_phi3/tokenizer_config.json',
 'tokenizer_phi3/special_tokens_map.json',
 'tokenizer_phi3/tokenizer.model',
 'tokenizer_phi3/added_tokens.json',
 'tokenizer_phi3/tokenizer.json')

In [15]:
import shutil

# Zip the model directory
shutil.make_archive("model_phi3", 'zip', "model_phi3")

# Zip the tokenizer directory
shutil.make_archive("tokenizer_phi3", 'zip', "tokenizer_phi3")

'/content/tokenizer_phi3.zip'

In [16]:
from google.colab import files

# Download the model
files.download("model_phi3.zip")

# Download the tokenizer
files.download("tokenizer_phi3.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>